# [LangChain](https://docs.langchain.com/docs/)

- LangChain is a framework for developing applications powered by language models.

## Topics Covered

Using LangChain to do the following:

1. Agents

## Installation

```sh
pip install langchain

# OR
pip install 'langchain[all]'

# Other dependencies
pip install python-dotenv
pip install openai
pip install -U wikipedia

```

In [1]:
# Built-in library
import itertools
import re
import json
from typing import Any, Dict, List, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
from pprint import pprint
import pandas as pd

# Visualization
import matplotlib.pyplot as plt


# pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black
# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate

from langchain.output_parsers import ResponseSchema, StructuredOutputParser

from langchain.chains import ConversationChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains import SequentialChain, SimpleSequentialChain
from langchain.chains import LLMChain


from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryBufferMemory,
    ConversationTokenBufferMemory,
    ConversationBufferWindowMemory,
)

from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
from IPython.display import display, Markdown


# Agents
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

#### Load Environment Variables

In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file

## LangChain Agents

In [4]:
# Init LLM
llm = ChatOpenAI(temperature=0)

# Load custom tools
tools = load_tools(["llm-math", "wikipedia"], llm=llm)

# Init agent: Load an agent executor given tools and LLM.
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True,
)

agent

AgentExecutor(memory=None, callbacks=None, callback_manager=None, verbose=True, tags=['chat-zero-shot-react-description'], metadata=None, agent=ChatAgent(llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=ChatPromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nWikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the nam

In [5]:
# Using the `llm-math` tool
pprint(agent("What's the square root of 625?"))



> Entering new AgentExecutor chain...
I can use the calculator tool to find the square root of 625.

Thought: I should use the calculator tool to find the square root of 625.
Action:
```
{
  "action": "Calculator",
  "action_input": "sqrt(625)"
}
```

Observation: Answer: 25.0
Thought:The square root of 625 is 25.0.
Final Answer: 25.0

> Finished chain.
{'input': "What's the square root of 625?", 'output': '25.0'}


In [6]:
# Using the `wikipedia` tool
question = """Ronnie Coleman is one of the greatest athlete. \
Which sport did he partake in and can you give me a short description of his career?"""

pprint(agent(question))



> Entering new AgentExecutor chain...
Thought: I need to find information about Ronnie Coleman's sport and his career.

Action:
```
{
  "action": "Wikipedia",
  "action_input": "Ronnie Coleman"
}
```
Observation: Page: Ronnie Coleman
Summary: Ronald Dean Coleman (born May 13, 1964) is an American retired professional bodybuilder. The winner of the Mr. Olympia title for eight consecutive years, he is widely regarded as either the greatest bodybuilder of all time or one of the two greatest along with Arnold Schwarzenegger and as the most dominant bodybuilding physique ever to grace the stage. Winner of 26 IFBB professional titles, he is also renowned for his combination of size and conditioning, dominant body-parts and extremely heavy workouts, making him the strongest bodybuilder of all time.For his dominance and legacy in the world of bodybuilding, Coleman was inducted into the International Sports Hall of Fame in 2016 and was bestowed with the 'Arnold Classic Lifetime Achievement Aw

#### Python Agent

In [7]:
python_agent = create_python_agent(llm, tool=PythonREPLTool(), verbose=True)
customer_list = [
    ["Neidu", "Emmanuel"],
    ["Dolly", "Too"],
    ["Lang", "Chain"],
    ["Eddie", "Murphy"],
    ["Jay", "Cutler"],
    ["Phil", "Heath"],
    ["Lionel", "Messi"],
]
question = f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}"""

pprint(python_agent.run(question))



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I can use the `sorted()` function to sort the list of customers. I will need to provide a key function that specifies the sorting order based on last name and then first name.
Action: Python_REPL
Action Input: sorted([['Neidu', 'Emmanuel'], ['Dolly', 'Too'], ['Lang', 'Chain'], ['Eddie', 'Murphy'], ['Jay', 'Cutler'], ['Phil', 'Heath'], ['Lionel', 'Messi']], key=lambda x: (x[1], x[0]))
Observation: 
Thought:The customers are sorted by last name and then first name.
Final Answer: [['Lang', 'Chain'], ['Jay', 'Cutler'], ['Phil', 'Heath'], ['Lionel', 'Messi'], ['Eddie', 'Murphy'], ['Neidu', 'Emmanuel'], ['Dolly', 'Too']]

> Finished chain.
("[['Lang', 'Chain'], ['Jay', 'Cutler'], ['Phil', 'Heath'], ['Lionel', "
 "'Messi'], ['Eddie', 'Murphy'], ['Neidu', 'Emmanuel'], ['Dolly', 'Too']]")


In [8]:
# View detailed outputs of the chains
import langchain

# Show internal logs
langchain.debug = True

pprint(python_agent.run(question))

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Neidu', 'Emmanuel'], ['Dolly', 'Too'], ['Lang', 'Chain'], ['Eddie', 'Murphy'], ['Jay', 'Cutler'], ['Phil', 'Heath'], ['Lionel', 'Messi']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Neidu', 'Emmanuel'], ['Dolly', 'Too'], ['Lang', 'Chain'], ['Eddie', 'Murphy'], ['Jay', 'Cutler'], ['Phil', 'Heath'], ['Lionel', 'Messi']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code

#### Custom Tools

```text
Define your own custom tools.
```

```sh  
pip install DateTime
```

In [9]:
from langchain.agents import tool
from datetime import date


@tool
def time() -> str:
    """This returns todays date and can be used for any questions
    related to knowing todays date. The input should always be an empty string,
    and this function will always return today's date.

    Note:
    Any date mathmatics should occur outside this function.
    """
    return str(date.today())

In [ ]:
# Note:
# The agent will sometimes come to the wrong conclusion (agents are a work in progress!).
# If it does, please try running it again.

try:
    result = agent("whats the date today?")
    pprint(result)
except:
    print("exception on external access")